In [64]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.feature_selection import SelectKBest, f_regression

In [65]:
df = pd.read_csv('../../datasets/processed_data/combined_features/TV18BRDCST.BO.csv')

In [66]:
df.columns

Index(['date', 'open', 'close', 'adj close', 'volume', 'low', 'high',
       'volume_adi', 'volume_obv', 'volume_cmf',
       ...
       'inr=x_percent_change', 'cl=f_percent_change',
       'treasury_yeild_10_years_percent_change', 'usdx-index_percent_change',
       '^nsei_percent_change', '^bsesn_percent_change', '^gspc_percent_change',
       'hsi_percent_change', 'sha_percent_change', '^sti_percent_change'],
      dtype='object', length=635)

In [67]:
def create_custom_target(df: pd.DataFrame) -> pd.DataFrame:
    """
    This function creates the custom target price, which is computed as ln(high/yesterday_close)
    """    
    # make a copy of the dataframe so as not to change the original dataframe
    data_df = df.copy()
    
    # create yesterday_close feature as
    data_df['yesterday_close'] = data_df['close'].shift(1)
    
    # create custom target price to predict, computing  ln(high/yesterday_close)
    data_df['ln_target'] = np.log(data_df['high'] / data_df['yesterday_close'])
    
    # as yesterday close would not be available for first day, 
    # we would not have custom target price for that day, which needs to be excluded 
    return(data_df.iloc[1:, ])

In [68]:
df = create_custom_target(df)

In [69]:
df['yesterday_close']

1       110.349998
2       107.349998
3       101.000000
4       100.800003
5        98.599998
           ...    
4016     34.220001
4017     34.220001
4018     34.360001
4019     34.220001
4020     35.270000
Name: yesterday_close, Length: 4020, dtype: float64

In [70]:
df['ln_target'] 

1       0.001358
2       0.013417
3       0.013278
4       0.037961
5      -0.037194
          ...   
4016    0.044579
4017    0.022538
4018    0.026707
4019    0.038130
4020    0.007907
Name: ln_target, Length: 4020, dtype: float64

In [71]:
# Drop cols with NaN values
df = df.dropna(axis=1, how='all')
df.fillna(method='ffill', inplace=True)

In [72]:
combined_date_df = df['date']
train_date, test_date = train_test_split(combined_date_df, train_size=0.8, shuffle=False)
dev_date, test_date = train_test_split(test_date, train_size=0.5, shuffle=False)

In [73]:
df_without_date = df.drop(columns=["date"])

In [74]:
df_without_date.dropna(inplace=True)

In [75]:
def timeseries_to_supervise(df, window_size, target):    
    X = []
    y = []
    indx = []
    no_records = len(df)
    #     
    for i in range(window_size, no_records):
        X.append(df.iloc[i-window_size:i].drop(target, axis=1).values.flatten())  # Collect past records as a sequence
        y.append(df.iloc[i][target])  # Next record as target variable
        indx.append(np.arange(i-window_size, i))

    X = pd.DataFrame(X)
    y = pd.Series(y)
    return(X, y, indx)

In [76]:
window_size = 10

In [77]:
# do train/test split the data with shuffle = False
train_data, test_data = train_test_split(df_without_date, train_size=0.8, shuffle=False)

# convert timeseries to be used in supervise learning model
X_train, y_train, indx_train = timeseries_to_supervise(train_data, window_size, 'ln_target')  

# further split test set to have an hold out set to be used for backtesting
dev_data, test_data = train_test_split(test_data, train_size=0.5, shuffle=False)

# convert timeseries to be used in supervise learning model    
X_test, y_test, indx_test = timeseries_to_supervise(dev_data, window_size, 'ln_target')  

In [78]:
selector = SelectKBest(score_func=f_regression, k=50)
X_train_50 = selector.fit_transform(X_train, y_train)
X_test_50 = selector.transform(X_test)

In [79]:
# Train the model on the top 50 features
model_50 = RandomForestRegressor(
    n_estimators=200,
    min_samples_split=2,
    min_samples_leaf=2,
    bootstrap=True
)
model_50.fit(X_train_50, y_train)

RandomForestRegressor(min_samples_leaf=2, n_estimators=200)

In [80]:
def convert_custom_target_to_actual_for_supervise(df: pd.DataFrame, window: int, y: "pd.Series[int]") -> "pd.Series[int]":
    """
    this module converts custom target - ln(high/yesterday_close) to actual high price again for timeseries converted data using rolling         window of size 10
    """
    data_df = df.copy()
    
    # exclude first 10 rows of train/test data, as while us
    
    y = np.exp(y) * data_df.loc[data_df.index[window:], 'yesterday_close'].reset_index(drop=True)
    return(y)    

In [81]:
def evaluate_model(model, window, dev_data, dev_date, X_test, y_test):
    
    # do target prediction using the provide model
    y_pred = model.predict(X_test)

    # convert back to original value, before computing mape            
    y_test = convert_custom_target_to_actual_for_supervise(dev_data, window, y_test)
    y_pred = convert_custom_target_to_actual_for_supervise(dev_data, window, y_pred)

    dev_dates = dev_date[window:].reset_index(drop=True)
    predictions_df = pd.DataFrame({'date': dev_dates, 'y_test': y_test, 'y_pred': y_pred})

    # compute regression metric - mape 
    mape = mean_absolute_percentage_error(y_test, y_pred)

    # compute rmse metric
    rmse = mean_squared_error(y_test, y_pred, squared=False)        
    return(predictions_df, mape, rmse)

In [82]:
predictions_df, mape, rmse = evaluate_model(model_50, window_size, dev_data, dev_date, X_test_50, y_test)

In [83]:
predictions_df, mape, rmse

(           date     y_test     y_pred
 0    2020-10-29  29.500000  30.699795
 1    2020-10-30  29.500000  29.783818
 2    2020-11-02  29.900000  29.296930
 3    2020-11-03  28.400000  28.546602
 4    2020-11-04  28.299999  28.660985
 ..          ...        ...        ...
 387  2022-02-04  67.849998  67.722493
 388  2022-02-07  65.699997  67.733879
 389  2022-02-08  64.250000  66.134236
 390  2022-02-09  70.000000  64.979680
 391  2022-02-09  70.000000  72.315044
 
 [392 rows x 3 columns],
 0.018668742134835378,
 1.335702484369603)

In [85]:
print(f"MAPE for model with top 50 features: {mape*100:.2f}%")

MAPE for model with top 50 features: 1.87%


In [86]:
import altair as alt

alt.themes.enable('fivethirtyeight')

predictions_df['date'] = pd.to_datetime(predictions_df['date'])

predictions_df['label'] = 'Actual'
predictions_df['predicted_label'] = 'Predicted'

# Actual high price line
line1 = alt.Chart(predictions_df).mark_line(strokeWidth=2).encode(
    x='date:T',
    y=alt.Y('y_test:Q', title='Price', scale=alt.Scale(zero=False)),
    color=alt.Color('label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Predicted high price line
line2 = alt.Chart(predictions_df).mark_line(strokeWidth=1, strokeDash=[3, 3]).encode(
    x='date:T',
    y=alt.Y('y_pred:Q', title='', scale=alt.Scale(zero=False)),
    color=alt.Color('predicted_label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Combine the two lines
chart = alt.layer(line1, line2).properties(
    title='Actual vs Predicted High Prices',
    width=650,
    height=400
).interactive()

chart

alt.LayerChart(...)

Repeat the process for the top 50 features without sentiment of TV18BRDCST.BO

In [50]:
df_without_sentiment = df.drop(columns=['agg_polarity', 'agg_compound', 'topic_polarity', 'topic_compound', 'ticker_polarity', 'ticker_compound'])

In [51]:
df_without_sentiment = create_custom_target(df_without_sentiment)

In [52]:
# Drop cols with NaN values
df_without_sentiment = df_without_sentiment.dropna(axis=1, how='all')
df_without_sentiment.fillna(method='ffill', inplace=True)

In [53]:
combined_date_df = df_without_sentiment['date']
train_date, test_date = train_test_split(combined_date_df, train_size=0.8, shuffle=False)
dev_date, test_date = train_test_split(test_date, train_size=0.5, shuffle=False)

In [54]:
df_without_date = df_without_sentiment.drop(columns=["date"])

In [55]:
df_without_date.dropna(inplace=True)

In [56]:
# do train/test split the data with shuffle = False
train_data, test_data = train_test_split(df_without_date, train_size=0.8, shuffle=False)

# convert timeseries to be used in supervise learning model
X_train, y_train, indx_train = timeseries_to_supervise(train_data, window_size, 'ln_target')  

# further split test set to have an hold out set to be used for backtesting
dev_data, test_data = train_test_split(test_data, train_size=0.5, shuffle=False)

# convert timeseries to be used in supervise learning model    
X_test, y_test, indx_test = timeseries_to_supervise(dev_data, window_size, 'ln_target')  

In [57]:
selector = SelectKBest(score_func=f_regression, k=50)
X_train_50 = selector.fit_transform(X_train, y_train)
X_test_50 = selector.transform(X_test)

In [58]:
# Train the model on the top 50 features
model_50 = RandomForestRegressor(
    n_estimators=200,
    min_samples_split=2,
    min_samples_leaf=2,
    bootstrap=True
)
model_50.fit(X_train_50, y_train)

RandomForestRegressor(min_samples_leaf=2, n_estimators=200)

In [59]:
predictions_df, mape, rmse = evaluate_model(model_50, window_size, dev_data, dev_date, X_test_50, y_test)

In [60]:
predictions_df, mape, rmse

(           date     y_test     y_pred
 0    2020-10-29  29.500000  30.680769
 1    2020-10-30  29.500000  29.751698
 2    2020-11-02  29.900000  29.341476
 3    2020-11-03  28.400000  28.555268
 4    2020-11-04  28.299999  28.630272
 ..          ...        ...        ...
 387  2022-02-04  67.849998  67.887173
 388  2022-02-07  65.699997  67.831643
 389  2022-02-08  64.250000  66.213474
 390  2022-02-09  70.000000  64.944593
 391  2022-02-09  70.000000  72.371382
 
 [392 rows x 3 columns],
 0.018728576892992125,
 1.3375543803740733)

In [42]:
print(f"MAPE for model with top 50 features without sentiment features: {mape*100:.2f}%")

MAPE for model with top 50 features without sentiment features: 1.87%


In [43]:
import altair as alt

alt.themes.enable('fivethirtyeight')

predictions_df['date'] = pd.to_datetime(predictions_df['date'])

predictions_df['label'] = 'Actual'
predictions_df['predicted_label'] = 'Predicted'

# Actual high price line
line1 = alt.Chart(predictions_df).mark_line(strokeWidth=2).encode(
    x='date:T',
    y=alt.Y('y_test:Q', title='Price', scale=alt.Scale(zero=False)),
    color=alt.Color('label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Predicted high price line
line2 = alt.Chart(predictions_df).mark_line(strokeWidth=1, strokeDash=[3, 3]).encode(
    x='date:T',
    y=alt.Y('y_pred:Q', title='', scale=alt.Scale(zero=False)),
    color=alt.Color('predicted_label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Combine the two lines
chart = alt.layer(line1, line2).properties(
    title='Actual vs Predicted High Prices',
    width=650,
    height=400
).interactive()

chart

alt.LayerChart(...)